2019-08-06 11:19:34  

# The evolution of compositionality under environmental noise



## Hypothesis:

**We hypothesise that** other-initiated repair and a compositional language co-evolve under the following assumptions:

1. There is a pressure for communicative success
2. Listeners have at least two types of repair initiator at their disposal: (i) open request, and (ii) restricted request
3. There is a pressure to make repair sequences as efficient as possible (in terms of combined utterance length of repair initiator and response)
    
This hypothesis is based on the idea that if repair sequences are under a pressure to be efficient, then it becomes useful to be able to initiate repair by asking for clarification of just _part_ of the utterance, specifically by feeding back the part of the meaning that you _did_ get. Under those circumstances, a compositional system is helpful (compared to a holistic system), because parts of the signal (i.e. 'substrings') map to parts of the meaning. _**<span class="girk">Note</span>**_ that this hypothesis is thus based on the assumption that the 'closed request' type of repair initiator works only if the listener has _understood_ part of the _meaning_ of the signal, as opposed to just having received part of the _signal_ and being able to repeat that back verbatim, in order to prompt the speaker to repeat the part that didn't come through. We feel that this is a reasonable assumption based on what we know from empirical work, but it is a little hard to assess given that natural language _is_ compositional.

There are of course parts of language that are holistic though. Mark D. and I thought of the example of proper nouns. For instance, if A says "My favourite writer is Chimamanda Ngozi Adichie", B could say "Chima _who_ ?" in order to prompt A to repeat the name. However, our intuition is that it would be quite unnatural for A to then respond to B's repair request by saying only "manda Ngozi Adichie". Most likely A would simply repeat the whole name.

When I was talking to Simon Kirby about it being difficult to think about whether repair could work as efficiently in a holistic system, he thought of colour terms as an example of a holistic part of language. And as an example he gave the repair sequence: 

A: "the blue shirt"

B: "you mean the turquoise one?"

In this example, the issue with the holistic signal "blue" is not that part of it was covered by noise, but that it is (apparently) not specific enough to pick out one particular referent. So it's not quite the same as what we were thinking about above: which is the question whether one can do efficient repair (i.e. a restricted request) when given a holistic signal that was partly disturbed by noise, in such a way that the listener does not get even _part_ of the meaning. Whereas in the example above, the listener receives the full signal without noise, and therefore is able to interpret part of the meaning (i.e. exclude some potential referents, such as the red shirt), but the problem is that they cannot quite narrow it down to a single unambiguous referent.

2019-08-06 11:19:27 

## Predictions:

Here we adapt the model of Kirby et al. (2015)---where they show that a compositional language evolves when there is a pressure for both (i) learnability and (ii) expressivity---to incorporate noise (step 1) and repair (step 2).

In Kirby et al.'s model, the pressure for expressivity (implemented as speakers aiming to avoid ambiguity when they produce a signal) is what causes populations to move towards languages that have one-to-one mappings between meanings and signals, but these can be either compositional or holistic. The pressure for learnability then (implemented as a prior that favours more compressible languages) is what causes populations to move towards compositional languages rather than holistic languages, because compositional languages are more compressible (and therefore, is the assumption, easier to learn). 

### Strong prediction (excluding pressure for learnability):

Therefore, the strongest prediction given our reasoning above, would be that if we take away the pressure for learnability, but add in environmental noise and the possibility of repair (combined with a pressure for efficient repair sequences), populations would also converge on compositional languages. This may be a bit of a weird prediction in itself, because we _do_ believe a pressure for learnability will have played a role. However, by isolating the effects of the two pressures (learnability vs. noise+repair) we could say something about their individual contributions.

### 'Weaker' prediction (including pressure for learnability):

Not entirely sure what to predict here. Maybe the effects of a noise+repair+efficiency pressure on the one hand, and a pressure for learnability on the other hand, will be additive? So that when combined, they cause populations to converge on a compositional language faster than when only one of these pressures is at play?


### Which predictions to test?

We'd like to explore both predictions, starting with the stronger one.

### References  
Kirby, S., Tamariz, M., Cornish, H., & Smith, K. (2015). Compression and communication in the cultural evolution of linguistic structure. Cognition, 141, 87–102. https://doi.org/10.1016/j.cognition.2015.03.016

2019-08-26 17:29:31 

### Question: How to implement the pressure for communicative success that we have in mind?

What I'm wondering here is whether the pressure for expressivity, as it was implemented by Kirby et al. (2015), in combination with a pressure for efficiency (and the possibility of repair), will have the effect of a pressure for communicative success as we have in mind, or whether we'll have to incorporate an extra or different pressure for communicative success.

Firstly, the way the pressure for expressivity is implemented by Kirby et al. (2015):
speakers try to avoid using ambiguous signals. ---> this is sufficient for pushing populations towards having either holistic or compositional languages (as opposed to degenerate ones). 

---> What could the effect of environmental noise be in this? If data consists of <meaning, form> pairs that were produced by a pair of agents from the previous generation, as in the Kirby et al. (2015) model, noise _without_ repair will mean that learners sometimes receive incomplete data (i.e. an incomplete form). Meanings in the <meaning, form> pairs that make up the data should not be incomplete, because (if I understand the Kirby et al., 2015, paper correctly) these represent the speaker's communicative intention, rather than the listener's interpretation.

---> What will that mean for how likely learners are to infer different languages? In general we'd just expect that learners will need more observations in order to reach a given amount of posterior belief in the correct language hypothesis. This means that given a particular bottleneck width, populations who are exposed to noise will be more likely to transition from one language into another than populations who aren't. Will the addition of noise push learners in any particular direction though? I can't think of why that would be the case if learners have a flat prior. However, if learners have a compression-based prior as in the Kirby et al. (2015) model, they would be pushed towards degenerate languages in general, in the same way as it happens in Kirby et al.

---> The pressure for communicative success that Mark D. and I had in mind (I think) is that speaker and hearer try to cooperate to reach mutual understanding; that is, so that the speaker can get their intended meaning across. **<span class="mark">Design decision:</span>** I think this pressure is captured if we assume that the hearer will initiate repair whenever (i) they are aware that noise happened, and (ii) they can't derive a single unambiguous meaning based on the form they received. The first condition is a bit artificial I think, because in natural language people do initiate repair even if they have received the full signal perfectly, but can't disambiguate the meaning of (some part of) it. However, given the simplified model of language that we're working with, it would also feel a bit artifical to make the listener initiate repair whenever they cannot map to form they received unambiguously to a single meaning, even if no noise was present, because in the absence of noise, the speaker doesn't actually have any mechanism to disambiguate <span class="burk">--> actually, this is not entirely true: if we stay as close as possible to the Kirby et al. (2015) model, and $\gamma > 0$, then the speaker does have a mechanism for disambiguating...</span> 
In some cases, having these two conditions for when to initiate repair will mean that a hearer will not have to initiate repair even when they received an incomplete (i.e. noisy) form. That is, if the hearer is using a language of the 'other' type, in which there is only one form that starts with a 'b', and that form maps to one particular meaning, then they will be able to infer a full meaning, and will therefore not initiate repair. However, such a language will necessarily also contain some degenerate forms, such as 'aa' mapping to both meaning '02' and meaning '03'. _That_ is where the pressure for expressivity comes in: speakers will try to avoid using such ambiguous forms, and therefore learners will be more likely to infer either a holistic or compositional language when receiving data from such a language of the 'other' type.

---> So in our 'strong prediction' scenario, where we _replace_ Kirby et al.'s learnability pressure with a pressure for efficient repair, we'd expect that populations get pushed in the direction of languages of the 'other' type, as well as languages of the compositional type rather than languages of the degenerate type. I'm not sure which language type would come out as best fit to a pressure for efficient repair however: 'other' or 'compositional'. So let's work through an example: 

2019-08-26 17:43:38 

### Worked example to figure out how each language type would fare under pressure for efficient repair.

Here's a language of the 'other' type which might be specifically useful under a pressure for efficient repair:
- 02 --> aa
- 03 --> ab
- 12 --> aa
- 13 --> ba

1. Noisy form a_ --> open request (because a_ does not map reliably to either meaning 0_ or 1_)
2. Noisy form b_ --> no repair (because it maps unambiguously to meaning 13)
3. Noisy form \_a --> open request (because \_a does not map reliably to either meaning \_2 or \_3)
4. Noisy form \_b --> no repair (because it maps unambiguously to meaning 03)

So all in all, a listener using this language would initiate the more costly form of repair in 50% of the cases where there is noise, and no repair at all in 50% of the cases where there's noise.

Now let's compare this to a case of a listener who uses a compositional language, such as this one:
- 02 --> aa
- 03 --> ab
- 12 --> ba
- 13 --> bb

1. Noisy form a_ --> restricted request (because a_ maps unambiguously to meaning 0_)
2. Noisy form b_ --> restricted request (because b_ maps unambiguously to meaning 0_)
3. Noisy form \_a --> restricted request (because \_a maps unambiguously to meaning \_2)
4. Noisy form \_b --> restricted request (because \_b maps unambiguously to meaning \_3)

So which language type allows for the most efficient repair depends on how costly we assume open requests are compared to restricted requests. If, for simplicity's sake, we assume an open request is twice as costly as a restricted request (say c=1.0 for an open request, c=0.5 for a restricted request, and c=0.0 for no repair), a compositional language would be _exactly_ as efficient as the other-type language above. Specifically, the average cost of repair for the other-type language would be: $(0.25*1.0)+(0.25*0.0)+(0.25*1.0)+(0.25*0.0) = 0.5$. And the average cost of repair for the compositional language would be: $(0.25*0.5)+(0.25*0.5)+(0.25*0.5)+(0.25*0.5) = 0.5$.

However, if we assume that there is some baseline cost for doing any repair at all (which seems more reasonable to me?), and that the cost of an open versus a restricted request are relatively closer together than the cost of a restricted request versus no repair at all, the other-type language above would win from the compositional language in terms of repair efficiency. To give a simple example, imagine that no repair costs 0.0, a restricted request costs 0.75, and an open request costs 1.0. Then the average cost of repair for the other-type language would still be: $(0.25*1.0)+(0.25*0.0)+(0.25*1.0)+(0.25*0.0) = 0.5$, whereas the average cost of repair of the compositional language would now be: $(0.25*0.75)+(0.25*0.75)+(0.25*0.75)+(0.25*0.75) = 0.75$.

<span class="mark">Perhaps we could base how we parameterise these costs on the empirical findings of the Dingemanse et al. (2015) paper in PLoS ONE, where they operationalise repair cost as the total utterance length of the repair sequence?</span>

<span class="mark">**Importantly however**</span>, this analysis does depend on the assumption that the listener is aware of whether noise occured or not, and only initiates repair when that is the case, not for disambiguation in general. If listeners also initiated repair for _general_ disambiguation purposes, in the absence of noise (even though their language has a homonym in it, which one could assume the listener could be aware of?), a listener who uses the 'other' type language above would also initiate repair _whenever_ they receive the 'aa' form. This would mean they would also initiate repair 50% of the time in the _absence_ of noise, making the overall rate of repair higher for languages of the 'other' type than for languages of the 'compositional' type.


Now let's work through a similar example comparing languages of the 'holistic' and 'compositional' type:
First, an example of a holistic language:
- 02 --> aa
- 03 --> bb
- 12 --> ba
- 13 --> ab

Here, the listener would also have to initiate repair in all cases where there is partial noise, just like with a compositional language. However, repair is less efficient because even if noise is only partial, the listener will have to use an open request in all cases, because they cannot make it more specific, because they don't have any part of the meaning.

Combining our pressure for efficient repair with Kirby et al.'s pressure for expressivity, should then get rid of the languages of the 'other' type. However, the extent to which this happens should depend on the relative strengths of the pressure for expressivity and the pressure for efficiency.2019-08-26 17:29:24 

---> So I guess what I'm saying is that in our model, the pressure for communicative success should be extended somewhat, to not only consist of the (parameterised) assumption that speakers try to avoid ambiguity, but also the assumption that listeners try to arrive at a complete, unambiguous interpretation, rather than just settling for a (wholly or partly) ambiguous meaning.

## How and where to implement efficiency pressure on repair?

It makes most sense that this pressure would be at play during communication, rather than during learning. We can't assume that learners prefer languages that allow for more efficient repair because they have some kind of a priori awareness of how efficient each of the languages would be in the case that repair is needed.

Therefore, instead of being located in the learner's prior, as the learnability pressure in the Kirby et al. (2015) model, the efficiency pressure should act somewhere during communication, just like Kirby et al.'s expressivity pressure. We could start by creating a simple cost function, where an open request is more costly than a restricted request (and exactly how much more costly it is could be regulated by some parameter?), and we could make the listener's process of whether or not to initiate repair a stochastic one, where they are less likely to initiate repair the more costly the repair sequence is. Because a compositional language would only ever require a restricted repair request, whereas a holistic language would only ever require an open request, 

2019-08-26 17:29:10 2019-07-31 15:27:33 

## Production in the Kirby et al. (2015) model:

$$
P(f \mid l, t) \propto \Bigg\{
\begin{array}{ll}
(\frac{1}{a})^\gamma \, (1-\epsilon) \; \; \textrm{if} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l\\ 
\frac{\epsilon}{|F|-1} \quad \quad \quad \textrm{if} \; t \; \textrm{is not mapped to} \; f \; \textrm{in} \; l
\end{array}
$$

where $f$ stands for form (in the sense of a complete signal, such as *aa*), $l$ stands for language, $t$ stands for topic, $a$ stands for ambiguity (i.e. the number of meanings in $M$ that map to form $f$ in $l$), $\gamma$ specifies the extent to which ambiguous utterances are penalised, and $\epsilon$ stands for production error.


From Kirby et al. (2015, pp. 92-93):
> "If $a$ = 1 ($f$ is unambiguous) and/or $\gamma$ = 0 then this yields a model of production where the 'correct' form is produced with probability $1-\epsilon$. However, when $\gamma$ > 0 and $f$ is ambiguous (i.e., $a$ > 1), then the 'correct' mapping from $t$ to $f$ is less likely to be produced (the probability $P(f \mid l, t)$ is reduced by the factor $(\frac{1}{a})^\gamma$ ) and the remaining probability mass is spread equally over the other possible forms, leading to increased probability of producing $f'$ $\neq$ $f$. Therefore, $\gamma$ > 0 introduces a penalty for languages whose utterances are ambiguous."


2019-07-31 15:27:52 
## How do we add *environmental* noise to this production model?


If we leave Kirby et al.'s production error $\epsilon$ aside for a moment, we could add environmental noise by allowing production to have two possible outcomes. Given a particular topic *t*, a speaker could produce either:
- form $f$ if $t$ is mapped to $f$ in $l$, with probability $1-n$ (where $n$ stands for the probability of noise happening)
- form $f_{noisy}$, where $f_{noisy}$ is a 'noisy variant' of $f$ *and* $t$ is mapped to $f$ in $l$. This should happen with probability $\frac{n}{|F_{noisy}|}$ (where $F_{noisy}$ is the full set of possible noisy variants of the form $f$ that maps to $t$ in $l$)

To give an example, using the example compositional grammar from Kirby et al. (2015, p. 92):

S   --> A B   
A:0 --> a   
A:1 --> b   
B:2 --> a   
B:3 --> b   

Given this grammar, the only form that maps to topic 02 is *aa*.   
_**<span class="mark"><span class="mark">Design decision:</span></span>**_ If we allow for both 'partial' and 'full' noise,   
 this form has three possible noisy variants:
- a_
- _a
- \__

where _ stands for a noisy part of the signal that the listener could not perceive.

<span class="girk">_**Decision**_ Mark D. + Marieke (01/08/2019):</span> Only partial noise is enough for now. We do want to allow for both open and restricted repair requests being a possibility, because the intuition that our hypothesis is based on, is that whereas a compositional system allows for the usage of both open and restricted requests, a holistic system only allows for open requests. However, to allow for open requests we don't necessarily need a form that is entirely noise (i.e. \_\_); we only need to assume that if the listener is able to disambiguate _part_ of the meaning, they use a restricted request, and if they cannot disambiguate any of the meaning they us an open request.

So if 02 is the speaker's intended topic (and if we exclude the possibility of the speaker making a production error for now), the speaker will produce the following signals with the following probabilities:
- aa with probability $1-n$
- [a_, \_a] with probability $\frac{n}{2}$ (because there are 2 possible noisy variants)

Now let's add the probability of making a production error $\epsilon$ back in, which gives us: 



$$
P(f \mid l, t) \propto \Bigg\{
\begin{array}{ll}
(\frac{1}{a})^\gamma \, (1-\epsilon) \, (1-n) \; \; \textrm{if} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is intact} \\ 
(\frac{1}{a})^\gamma \, (1-\epsilon) \, \frac{n}{|F_{noisy}|} \quad \quad \; \; \textrm{if part of} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is not intact} \\ 
\frac{\epsilon}{|F|-1} \, (1-n) \quad \quad \quad \textrm{if} \; t \; \textrm{is not mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is intact}\\
\frac{\epsilon}{|F|-1} \, \frac{n}{|F_{noisy}|} \quad \quad \quad \quad \quad \textrm{if no part of} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l \; \textrm{and} \; f \; \textrm{is not intact}\\
\end{array}
$$

where *n* stands for the probability of noise.

<span class="burk">Actually, the probability for when f is _not_ mapped to t, _and_ f is a noisy variant, is not quite as simple as stated above.</span> Depending on how many forms f there are that _don't_ map to t, some of those error forms will generate the same noisy variant. For example, if the form that _is_ mapped to the intended topic is 'aa', and the forms ['ab', 'ba', 'bb'] form the set of forms that _aren't_ mapped to t, some noisy variants will only occur given one of these error forms being what the speaker produced ('a_' in this case), while others will occur given multiple possible error forms that the speaker might produce (e.g. '\_b' can be a noisy variant of either 'ab' or 'bb'). The probabilities for the different noisy variants should reflect this. (I.e. in the example above, noisy variant '\_b' should be twice as probable as noisy variant 'a_'.)

2019-07-31 15:28:14 

## First, let's reproduce Kirby et al.'s (2015) model of production:

In order to do that we first need to set some general parameters:

In [1]:
meanings = ['02', '03', '12', '13']  # all possible meanings
forms = ['aa', 'ab', 'ba', 'bb']  # all possible forms
error = 0.05  # the probability of making a production error; 
                # 0.05 is the value Kirby et al. (2015) used in the body of their paper


Then we need some functions to generate all possible languages and classify languages according to the categories specified by Kirby et al. (2015):

In [3]:
import itertools
import numpy as np

# Some functions to create and classify all possible languages:
def create_all_possible_languages(meanings, forms):
    """Creates all possible languages

    :param meanings: list of strings corresponding to all possible meanings
    :type meanings: list
    :param forms: list of strings corresponding to all possible forms
    :type forms: list
    :returns: list of tuples which represent languages, where each tuple consists of forms and has length len(meanings)
    :rtype: list
    """
    all_possible_languages = list(itertools.product(forms, repeat=len(meanings)))
    return all_possible_languages


def classify_language(lang, forms, meanings):
    """
    Classify one particular language as either 'degenerate' (0), 'holistic' (1), 'other' (2)
    or 'compositional' (3) (Kirby et al., 2015)

    :param lang: a language; represented as a tuple of forms, where each form index maps to same index in meanings
    :type lang: tuple
    :param forms: list of strings corresponding to all possible forms
    :type forms: list
    :returns: integer corresponding to category that language belongs to:
    0 = degenerate, 1 = holistic, 2 = other, 3 = compositional (here I'm following the
    numbering used in SimLang lab 21)
    :rtype: int
    """
    # TODO: See if I can modify this function so that it can deal with any number of forms and meanings.
    class_degenerate = 0
    class_holistic = 1
    class_other = 2
    class_compositional = 3
    # First check whether some conditions are met, bc this function hasn't been coded up in the most general way yet:
    if len(forms) != 4:
        raise ValueError(
            "This function only works for a world in which there are 4 possible forms"
        )
    if len(forms[0]) != 2:
        raise ValueError(
            "This function only works when each form consists of 2 elements")
    if len(lang) != len(meanings):
        raise ValueError("lang should have same length as meanings")

    # lang is degenerate if it uses the same form for every meaning:
    if lang[0] == lang[1] and lang[1] == lang[2] and lang[2] == lang[3]:
        return class_degenerate

    # lang is compositional if it makes use of all possible forms, *and* each form element maps to the same meaning
    # element for each form:
    elif forms[0] in lang and forms[1] in lang and forms[2] in lang and forms[
        3] in lang and lang[0][0] == lang[1][0] and lang[2][0] == lang[3][0] and lang[0][
        1] == lang[2][1] and lang[1][1] == lang[3][1]:
        return class_compositional

    # lang is holistic if it is *not* compositional, but *does* make use of all possible forms:
    elif forms[0] in lang and forms[1] in lang and forms[2] in lang and forms[
        3] in lang:
        return class_holistic

    # In all other cases, a language belongs to the 'other' category:
    else:
        return class_other


def classify_all_languages(language_list):
    """
    Classify all languages as either 'degenerate' (0), 'holistic' (1), 'other' (2)
    or 'compositional' (3) (Kirby et al., 2015)

    :param language_list: list of all languages
    :type language_list: list
    :returns: 1D numpy array containing integer corresponding to category of corresponding
    language index: 0 = degenerate, 1 = holistic, 2 = other, 3 = compositional
    (here I'm following the numbering used in SimLang lab 21)
    :rtype: 1D numpy array
    """
    class_per_lang = np.zeros(len(language_list))
    for l in range(len(language_list)):
        class_per_lang[l] = classify_language(language_list[l], forms, meanings)
    return class_per_lang

In [10]:
# Let's try out our create_all_possible_languages() function:

all_possible_languages = create_all_possible_languages(meanings, forms)
# print("all_possible_languages are:")
# print(all_possible_languages)
print("number of possible languages is:")
print(len(all_possible_languages))

number of possible languages is:
256


In [11]:
# Let's test our classify_language() function using some example languages 
# from the Kirby et al. (2015) paper (and the numbering of classes used 
# in SimLang lab 21):

degenerate_lang = ('aa', 'aa', 'aa', 'aa')
print('')
print("degenerate_lang is:")
print(degenerate_lang)
class_degenerate_lang = classify_language(degenerate_lang, forms, meanings)
print("class_degenerate_lang is:")
print(class_degenerate_lang)

holistic_lang = ('aa', 'ab', 'bb', 'ba')
print('')
print("holistic_lang is:")
print(holistic_lang)
class_holistic_lang = classify_language(holistic_lang, forms, meanings)
print("class_holistic_lang is:")
print(class_holistic_lang)

other_lang = ('aa', 'aa', 'aa', 'ab')
print('')
print("other_lang is:")
print(other_lang)
class_other_lang = classify_language(other_lang, forms, meanings)
print("class_other_lang is:")
print(class_other_lang)

compositional_lang = ('aa', 'ab', 'ba', 'bb')
print('')
print("compositional_lang is:")
print(compositional_lang)
class_compositional_lang = classify_language(compositional_lang, forms,
                                             meanings)
print("class_compositional_lang is:")
print(class_compositional_lang)


degenerate_lang is:
('aa', 'aa', 'aa', 'aa')
class_degenerate_lang is:
0

holistic_lang is:
('aa', 'ab', 'bb', 'ba')
class_holistic_lang is:
1

other_lang is:
('aa', 'aa', 'aa', 'ab')
class_other_lang is:
2

compositional_lang is:
('aa', 'ab', 'ba', 'bb')
class_compositional_lang is:
3


Let's check whether the functions in this cell work correctly by comparing the number of languages of each type we get with the SimLang lab 21:

In [12]:
# COPIED FROM SIMLANG LAB 21:
languages = [[('02', 'aa'), ('03', 'aa'), ('12', 'aa'), ('13', 'aa')], [('02', 'aa'), ('03', 'aa'), ('12', 'aa'), ('13', 'ab')], [('02', 'aa'), ('03', 'aa'), ('12', 'aa'), ('13', 'ba')], [('02', 'aa'), ('03', 'aa'), ('12', 'aa'), ('13', 'bb')], [('02', 'aa'), ('03', 'aa'), ('12', 'ab'), ('13', 'aa')], [('02', 'aa'), ('03', 'aa'), ('12', 'ab'), ('13', 'ab')], [('02', 'aa'), ('03', 'aa'), ('12', 'ab'), ('13', 'ba')], [('02', 'aa'), ('03', 'aa'), ('12', 'ab'), ('13', 'bb')], [('02', 'aa'), ('03', 'aa'), ('12', 'ba'), ('13', 'aa')], [('02', 'aa'), ('03', 'aa'), ('12', 'ba'), ('13', 'ab')], [('02', 'aa'), ('03', 'aa'), ('12', 'ba'), ('13', 'ba')], [('02', 'aa'), ('03', 'aa'), ('12', 'ba'), ('13', 'bb')], [('02', 'aa'), ('03', 'aa'), ('12', 'bb'), ('13', 'aa')], [('02', 'aa'), ('03', 'aa'), ('12', 'bb'), ('13', 'ab')], [('02', 'aa'), ('03', 'aa'), ('12', 'bb'), ('13', 'ba')], [('02', 'aa'), ('03', 'aa'), ('12', 'bb'), ('13', 'bb')], [('02', 'aa'), ('03', 'ab'), ('12', 'aa'), ('13', 'aa')], [('02', 'aa'), ('03', 'ab'), ('12', 'aa'), ('13', 'ab')], [('02', 'aa'), ('03', 'ab'), ('12', 'aa'), ('13', 'ba')], [('02', 'aa'), ('03', 'ab'), ('12', 'aa'), ('13', 'bb')], [('02', 'aa'), ('03', 'ab'), ('12', 'ab'), ('13', 'aa')], [('02', 'aa'), ('03', 'ab'), ('12', 'ab'), ('13', 'ab')], [('02', 'aa'), ('03', 'ab'), ('12', 'ab'), ('13', 'ba')], [('02', 'aa'), ('03', 'ab'), ('12', 'ab'), ('13', 'bb')], [('02', 'aa'), ('03', 'ab'), ('12', 'ba'), ('13', 'aa')], [('02', 'aa'), ('03', 'ab'), ('12', 'ba'), ('13', 'ab')], [('02', 'aa'), ('03', 'ab'), ('12', 'ba'), ('13', 'ba')], [('02', 'aa'), ('03', 'ab'), ('12', 'ba'), ('13', 'bb')], [('02', 'aa'), ('03', 'ab'), ('12', 'bb'), ('13', 'aa')], [('02', 'aa'), ('03', 'ab'), ('12', 'bb'), ('13', 'ab')], [('02', 'aa'), ('03', 'ab'), ('12', 'bb'), ('13', 'ba')], [('02', 'aa'), ('03', 'ab'), ('12', 'bb'), ('13', 'bb')], [('02', 'aa'), ('03', 'ba'), ('12', 'aa'), ('13', 'aa')], [('02', 'aa'), ('03', 'ba'), ('12', 'aa'), ('13', 'ab')], [('02', 'aa'), ('03', 'ba'), ('12', 'aa'), ('13', 'ba')], [('02', 'aa'), ('03', 'ba'), ('12', 'aa'), ('13', 'bb')], [('02', 'aa'), ('03', 'ba'), ('12', 'ab'), ('13', 'aa')], [('02', 'aa'), ('03', 'ba'), ('12', 'ab'), ('13', 'ab')], [('02', 'aa'), ('03', 'ba'), ('12', 'ab'), ('13', 'ba')], [('02', 'aa'), ('03', 'ba'), ('12', 'ab'), ('13', 'bb')], [('02', 'aa'), ('03', 'ba'), ('12', 'ba'), ('13', 'aa')], [('02', 'aa'), ('03', 'ba'), ('12', 'ba'), ('13', 'ab')], [('02', 'aa'), ('03', 'ba'), ('12', 'ba'), ('13', 'ba')], [('02', 'aa'), ('03', 'ba'), ('12', 'ba'), ('13', 'bb')], [('02', 'aa'), ('03', 'ba'), ('12', 'bb'), ('13', 'aa')], [('02', 'aa'), ('03', 'ba'), ('12', 'bb'), ('13', 'ab')], [('02', 'aa'), ('03', 'ba'), ('12', 'bb'), ('13', 'ba')], [('02', 'aa'), ('03', 'ba'), ('12', 'bb'), ('13', 'bb')], [('02', 'aa'), ('03', 'bb'), ('12', 'aa'), ('13', 'aa')], [('02', 'aa'), ('03', 'bb'), ('12', 'aa'), ('13', 'ab')], [('02', 'aa'), ('03', 'bb'), ('12', 'aa'), ('13', 'ba')], [('02', 'aa'), ('03', 'bb'), ('12', 'aa'), ('13', 'bb')], [('02', 'aa'), ('03', 'bb'), ('12', 'ab'), ('13', 'aa')], [('02', 'aa'), ('03', 'bb'), ('12', 'ab'), ('13', 'ab')], [('02', 'aa'), ('03', 'bb'), ('12', 'ab'), ('13', 'ba')], [('02', 'aa'), ('03', 'bb'), ('12', 'ab'), ('13', 'bb')], [('02', 'aa'), ('03', 'bb'), ('12', 'ba'), ('13', 'aa')], [('02', 'aa'), ('03', 'bb'), ('12', 'ba'), ('13', 'ab')], [('02', 'aa'), ('03', 'bb'), ('12', 'ba'), ('13', 'ba')], [('02', 'aa'), ('03', 'bb'), ('12', 'ba'), ('13', 'bb')], [('02', 'aa'), ('03', 'bb'), ('12', 'bb'), ('13', 'aa')], [('02', 'aa'), ('03', 'bb'), ('12', 'bb'), ('13', 'ab')], [('02', 'aa'), ('03', 'bb'), ('12', 'bb'), ('13', 'ba')], [('02', 'aa'), ('03', 'bb'), ('12', 'bb'), ('13', 'bb')], [('02', 'ab'), ('03', 'aa'), ('12', 'aa'), ('13', 'aa')], [('02', 'ab'), ('03', 'aa'), ('12', 'aa'), ('13', 'ab')], [('02', 'ab'), ('03', 'aa'), ('12', 'aa'), ('13', 'ba')], [('02', 'ab'), ('03', 'aa'), ('12', 'aa'), ('13', 'bb')], [('02', 'ab'), ('03', 'aa'), ('12', 'ab'), ('13', 'aa')], [('02', 'ab'), ('03', 'aa'), ('12', 'ab'), ('13', 'ab')], [('02', 'ab'), ('03', 'aa'), ('12', 'ab'), ('13', 'ba')], [('02', 'ab'), ('03', 'aa'), ('12', 'ab'), ('13', 'bb')], [('02', 'ab'), ('03', 'aa'), ('12', 'ba'), ('13', 'aa')], [('02', 'ab'), ('03', 'aa'), ('12', 'ba'), ('13', 'ab')], [('02', 'ab'), ('03', 'aa'), ('12', 'ba'), ('13', 'ba')], [('02', 'ab'), ('03', 'aa'), ('12', 'ba'), ('13', 'bb')], [('02', 'ab'), ('03', 'aa'), ('12', 'bb'), ('13', 'aa')], [('02', 'ab'), ('03', 'aa'), ('12', 'bb'), ('13', 'ab')], [('02', 'ab'), ('03', 'aa'), ('12', 'bb'), ('13', 'ba')], [('02', 'ab'), ('03', 'aa'), ('12', 'bb'), ('13', 'bb')], [('02', 'ab'), ('03', 'ab'), ('12', 'aa'), ('13', 'aa')], [('02', 'ab'), ('03', 'ab'), ('12', 'aa'), ('13', 'ab')], [('02', 'ab'), ('03', 'ab'), ('12', 'aa'), ('13', 'ba')], [('02', 'ab'), ('03', 'ab'), ('12', 'aa'), ('13', 'bb')], [('02', 'ab'), ('03', 'ab'), ('12', 'ab'), ('13', 'aa')], [('02', 'ab'), ('03', 'ab'), ('12', 'ab'), ('13', 'ab')], [('02', 'ab'), ('03', 'ab'), ('12', 'ab'), ('13', 'ba')], [('02', 'ab'), ('03', 'ab'), ('12', 'ab'), ('13', 'bb')], [('02', 'ab'), ('03', 'ab'), ('12', 'ba'), ('13', 'aa')], [('02', 'ab'), ('03', 'ab'), ('12', 'ba'), ('13', 'ab')], [('02', 'ab'), ('03', 'ab'), ('12', 'ba'), ('13', 'ba')], [('02', 'ab'), ('03', 'ab'), ('12', 'ba'), ('13', 'bb')], [('02', 'ab'), ('03', 'ab'), ('12', 'bb'), ('13', 'aa')], [('02', 'ab'), ('03', 'ab'), ('12', 'bb'), ('13', 'ab')], [('02', 'ab'), ('03', 'ab'), ('12', 'bb'), ('13', 'ba')], [('02', 'ab'), ('03', 'ab'), ('12', 'bb'), ('13', 'bb')], [('02', 'ab'), ('03', 'ba'), ('12', 'aa'), ('13', 'aa')], [('02', 'ab'), ('03', 'ba'), ('12', 'aa'), ('13', 'ab')], [('02', 'ab'), ('03', 'ba'), ('12', 'aa'), ('13', 'ba')], [('02', 'ab'), ('03', 'ba'), ('12', 'aa'), ('13', 'bb')], [('02', 'ab'), ('03', 'ba'), ('12', 'ab'), ('13', 'aa')], [('02', 'ab'), ('03', 'ba'), ('12', 'ab'), ('13', 'ab')], [('02', 'ab'), ('03', 'ba'), ('12', 'ab'), ('13', 'ba')], [('02', 'ab'), ('03', 'ba'), ('12', 'ab'), ('13', 'bb')], [('02', 'ab'), ('03', 'ba'), ('12', 'ba'), ('13', 'aa')], [('02', 'ab'), ('03', 'ba'), ('12', 'ba'), ('13', 'ab')], [('02', 'ab'), ('03', 'ba'), ('12', 'ba'), ('13', 'ba')], [('02', 'ab'), ('03', 'ba'), ('12', 'ba'), ('13', 'bb')], [('02', 'ab'), ('03', 'ba'), ('12', 'bb'), ('13', 'aa')], [('02', 'ab'), ('03', 'ba'), ('12', 'bb'), ('13', 'ab')], [('02', 'ab'), ('03', 'ba'), ('12', 'bb'), ('13', 'ba')], [('02', 'ab'), ('03', 'ba'), ('12', 'bb'), ('13', 'bb')], [('02', 'ab'), ('03', 'bb'), ('12', 'aa'), ('13', 'aa')], [('02', 'ab'), ('03', 'bb'), ('12', 'aa'), ('13', 'ab')], [('02', 'ab'), ('03', 'bb'), ('12', 'aa'), ('13', 'ba')], [('02', 'ab'), ('03', 'bb'), ('12', 'aa'), ('13', 'bb')], [('02', 'ab'), ('03', 'bb'), ('12', 'ab'), ('13', 'aa')], [('02', 'ab'), ('03', 'bb'), ('12', 'ab'), ('13', 'ab')], [('02', 'ab'), ('03', 'bb'), ('12', 'ab'), ('13', 'ba')], [('02', 'ab'), ('03', 'bb'), ('12', 'ab'), ('13', 'bb')], [('02', 'ab'), ('03', 'bb'), ('12', 'ba'), ('13', 'aa')], [('02', 'ab'), ('03', 'bb'), ('12', 'ba'), ('13', 'ab')], [('02', 'ab'), ('03', 'bb'), ('12', 'ba'), ('13', 'ba')], [('02', 'ab'), ('03', 'bb'), ('12', 'ba'), ('13', 'bb')], [('02', 'ab'), ('03', 'bb'), ('12', 'bb'), ('13', 'aa')], [('02', 'ab'), ('03', 'bb'), ('12', 'bb'), ('13', 'ab')], [('02', 'ab'), ('03', 'bb'), ('12', 'bb'), ('13', 'ba')], [('02', 'ab'), ('03', 'bb'), ('12', 'bb'), ('13', 'bb')], [('02', 'ba'), ('03', 'aa'), ('12', 'aa'), ('13', 'aa')], [('02', 'ba'), ('03', 'aa'), ('12', 'aa'), ('13', 'ab')], [('02', 'ba'), ('03', 'aa'), ('12', 'aa'), ('13', 'ba')], [('02', 'ba'), ('03', 'aa'), ('12', 'aa'), ('13', 'bb')], [('02', 'ba'), ('03', 'aa'), ('12', 'ab'), ('13', 'aa')], [('02', 'ba'), ('03', 'aa'), ('12', 'ab'), ('13', 'ab')], [('02', 'ba'), ('03', 'aa'), ('12', 'ab'), ('13', 'ba')], [('02', 'ba'), ('03', 'aa'), ('12', 'ab'), ('13', 'bb')], [('02', 'ba'), ('03', 'aa'), ('12', 'ba'), ('13', 'aa')], [('02', 'ba'), ('03', 'aa'), ('12', 'ba'), ('13', 'ab')], [('02', 'ba'), ('03', 'aa'), ('12', 'ba'), ('13', 'ba')], [('02', 'ba'), ('03', 'aa'), ('12', 'ba'), ('13', 'bb')], [('02', 'ba'), ('03', 'aa'), ('12', 'bb'), ('13', 'aa')], [('02', 'ba'), ('03', 'aa'), ('12', 'bb'), ('13', 'ab')], [('02', 'ba'), ('03', 'aa'), ('12', 'bb'), ('13', 'ba')], [('02', 'ba'), ('03', 'aa'), ('12', 'bb'), ('13', 'bb')], [('02', 'ba'), ('03', 'ab'), ('12', 'aa'), ('13', 'aa')], [('02', 'ba'), ('03', 'ab'), ('12', 'aa'), ('13', 'ab')], [('02', 'ba'), ('03', 'ab'), ('12', 'aa'), ('13', 'ba')], [('02', 'ba'), ('03', 'ab'), ('12', 'aa'), ('13', 'bb')], [('02', 'ba'), ('03', 'ab'), ('12', 'ab'), ('13', 'aa')], [('02', 'ba'), ('03', 'ab'), ('12', 'ab'), ('13', 'ab')], [('02', 'ba'), ('03', 'ab'), ('12', 'ab'), ('13', 'ba')], [('02', 'ba'), ('03', 'ab'), ('12', 'ab'), ('13', 'bb')], [('02', 'ba'), ('03', 'ab'), ('12', 'ba'), ('13', 'aa')], [('02', 'ba'), ('03', 'ab'), ('12', 'ba'), ('13', 'ab')], [('02', 'ba'), ('03', 'ab'), ('12', 'ba'), ('13', 'ba')], [('02', 'ba'), ('03', 'ab'), ('12', 'ba'), ('13', 'bb')], [('02', 'ba'), ('03', 'ab'), ('12', 'bb'), ('13', 'aa')], [('02', 'ba'), ('03', 'ab'), ('12', 'bb'), ('13', 'ab')], [('02', 'ba'), ('03', 'ab'), ('12', 'bb'), ('13', 'ba')], [('02', 'ba'), ('03', 'ab'), ('12', 'bb'), ('13', 'bb')], [('02', 'ba'), ('03', 'ba'), ('12', 'aa'), ('13', 'aa')], [('02', 'ba'), ('03', 'ba'), ('12', 'aa'), ('13', 'ab')], [('02', 'ba'), ('03', 'ba'), ('12', 'aa'), ('13', 'ba')], [('02', 'ba'), ('03', 'ba'), ('12', 'aa'), ('13', 'bb')], [('02', 'ba'), ('03', 'ba'), ('12', 'ab'), ('13', 'aa')], [('02', 'ba'), ('03', 'ba'), ('12', 'ab'), ('13', 'ab')], [('02', 'ba'), ('03', 'ba'), ('12', 'ab'), ('13', 'ba')], [('02', 'ba'), ('03', 'ba'), ('12', 'ab'), ('13', 'bb')], [('02', 'ba'), ('03', 'ba'), ('12', 'ba'), ('13', 'aa')], [('02', 'ba'), ('03', 'ba'), ('12', 'ba'), ('13', 'ab')], [('02', 'ba'), ('03', 'ba'), ('12', 'ba'), ('13', 'ba')], [('02', 'ba'), ('03', 'ba'), ('12', 'ba'), ('13', 'bb')], [('02', 'ba'), ('03', 'ba'), ('12', 'bb'), ('13', 'aa')], [('02', 'ba'), ('03', 'ba'), ('12', 'bb'), ('13', 'ab')], [('02', 'ba'), ('03', 'ba'), ('12', 'bb'), ('13', 'ba')], [('02', 'ba'), ('03', 'ba'), ('12', 'bb'), ('13', 'bb')], [('02', 'ba'), ('03', 'bb'), ('12', 'aa'), ('13', 'aa')], [('02', 'ba'), ('03', 'bb'), ('12', 'aa'), ('13', 'ab')], [('02', 'ba'), ('03', 'bb'), ('12', 'aa'), ('13', 'ba')], [('02', 'ba'), ('03', 'bb'), ('12', 'aa'), ('13', 'bb')], [('02', 'ba'), ('03', 'bb'), ('12', 'ab'), ('13', 'aa')], [('02', 'ba'), ('03', 'bb'), ('12', 'ab'), ('13', 'ab')], [('02', 'ba'), ('03', 'bb'), ('12', 'ab'), ('13', 'ba')], [('02', 'ba'), ('03', 'bb'), ('12', 'ab'), ('13', 'bb')], [('02', 'ba'), ('03', 'bb'), ('12', 'ba'), ('13', 'aa')], [('02', 'ba'), ('03', 'bb'), ('12', 'ba'), ('13', 'ab')], [('02', 'ba'), ('03', 'bb'), ('12', 'ba'), ('13', 'ba')], [('02', 'ba'), ('03', 'bb'), ('12', 'ba'), ('13', 'bb')], [('02', 'ba'), ('03', 'bb'), ('12', 'bb'), ('13', 'aa')], [('02', 'ba'), ('03', 'bb'), ('12', 'bb'), ('13', 'ab')], [('02', 'ba'), ('03', 'bb'), ('12', 'bb'), ('13', 'ba')], [('02', 'ba'), ('03', 'bb'), ('12', 'bb'), ('13', 'bb')], [('02', 'bb'), ('03', 'aa'), ('12', 'aa'), ('13', 'aa')], [('02', 'bb'), ('03', 'aa'), ('12', 'aa'), ('13', 'ab')], [('02', 'bb'), ('03', 'aa'), ('12', 'aa'), ('13', 'ba')], [('02', 'bb'), ('03', 'aa'), ('12', 'aa'), ('13', 'bb')], [('02', 'bb'), ('03', 'aa'), ('12', 'ab'), ('13', 'aa')], [('02', 'bb'), ('03', 'aa'), ('12', 'ab'), ('13', 'ab')], [('02', 'bb'), ('03', 'aa'), ('12', 'ab'), ('13', 'ba')], [('02', 'bb'), ('03', 'aa'), ('12', 'ab'), ('13', 'bb')], [('02', 'bb'), ('03', 'aa'), ('12', 'ba'), ('13', 'aa')], [('02', 'bb'), ('03', 'aa'), ('12', 'ba'), ('13', 'ab')], [('02', 'bb'), ('03', 'aa'), ('12', 'ba'), ('13', 'ba')], [('02', 'bb'), ('03', 'aa'), ('12', 'ba'), ('13', 'bb')], [('02', 'bb'), ('03', 'aa'), ('12', 'bb'), ('13', 'aa')], [('02', 'bb'), ('03', 'aa'), ('12', 'bb'), ('13', 'ab')], [('02', 'bb'), ('03', 'aa'), ('12', 'bb'), ('13', 'ba')], [('02', 'bb'), ('03', 'aa'), ('12', 'bb'), ('13', 'bb')], [('02', 'bb'), ('03', 'ab'), ('12', 'aa'), ('13', 'aa')], [('02', 'bb'), ('03', 'ab'), ('12', 'aa'), ('13', 'ab')], [('02', 'bb'), ('03', 'ab'), ('12', 'aa'), ('13', 'ba')], [('02', 'bb'), ('03', 'ab'), ('12', 'aa'), ('13', 'bb')], [('02', 'bb'), ('03', 'ab'), ('12', 'ab'), ('13', 'aa')], [('02', 'bb'), ('03', 'ab'), ('12', 'ab'), ('13', 'ab')], [('02', 'bb'), ('03', 'ab'), ('12', 'ab'), ('13', 'ba')], [('02', 'bb'), ('03', 'ab'), ('12', 'ab'), ('13', 'bb')], [('02', 'bb'), ('03', 'ab'), ('12', 'ba'), ('13', 'aa')], [('02', 'bb'), ('03', 'ab'), ('12', 'ba'), ('13', 'ab')], [('02', 'bb'), ('03', 'ab'), ('12', 'ba'), ('13', 'ba')], [('02', 'bb'), ('03', 'ab'), ('12', 'ba'), ('13', 'bb')], [('02', 'bb'), ('03', 'ab'), ('12', 'bb'), ('13', 'aa')], [('02', 'bb'), ('03', 'ab'), ('12', 'bb'), ('13', 'ab')], [('02', 'bb'), ('03', 'ab'), ('12', 'bb'), ('13', 'ba')], [('02', 'bb'), ('03', 'ab'), ('12', 'bb'), ('13', 'bb')], [('02', 'bb'), ('03', 'ba'), ('12', 'aa'), ('13', 'aa')], [('02', 'bb'), ('03', 'ba'), ('12', 'aa'), ('13', 'ab')], [('02', 'bb'), ('03', 'ba'), ('12', 'aa'), ('13', 'ba')], [('02', 'bb'), ('03', 'ba'), ('12', 'aa'), ('13', 'bb')], [('02', 'bb'), ('03', 'ba'), ('12', 'ab'), ('13', 'aa')], [('02', 'bb'), ('03', 'ba'), ('12', 'ab'), ('13', 'ab')], [('02', 'bb'), ('03', 'ba'), ('12', 'ab'), ('13', 'ba')], [('02', 'bb'), ('03', 'ba'), ('12', 'ab'), ('13', 'bb')], [('02', 'bb'), ('03', 'ba'), ('12', 'ba'), ('13', 'aa')], [('02', 'bb'), ('03', 'ba'), ('12', 'ba'), ('13', 'ab')], [('02', 'bb'), ('03', 'ba'), ('12', 'ba'), ('13', 'ba')], [('02', 'bb'), ('03', 'ba'), ('12', 'ba'), ('13', 'bb')], [('02', 'bb'), ('03', 'ba'), ('12', 'bb'), ('13', 'aa')], [('02', 'bb'), ('03', 'ba'), ('12', 'bb'), ('13', 'ab')], [('02', 'bb'), ('03', 'ba'), ('12', 'bb'), ('13', 'ba')], [('02', 'bb'), ('03', 'ba'), ('12', 'bb'), ('13', 'bb')], [('02', 'bb'), ('03', 'bb'), ('12', 'aa'), ('13', 'aa')], [('02', 'bb'), ('03', 'bb'), ('12', 'aa'), ('13', 'ab')], [('02', 'bb'), ('03', 'bb'), ('12', 'aa'), ('13', 'ba')], [('02', 'bb'), ('03', 'bb'), ('12', 'aa'), ('13', 'bb')], [('02', 'bb'), ('03', 'bb'), ('12', 'ab'), ('13', 'aa')], [('02', 'bb'), ('03', 'bb'), ('12', 'ab'), ('13', 'ab')], [('02', 'bb'), ('03', 'bb'), ('12', 'ab'), ('13', 'ba')], [('02', 'bb'), ('03', 'bb'), ('12', 'ab'), ('13', 'bb')], [('02', 'bb'), ('03', 'bb'), ('12', 'ba'), ('13', 'aa')], [('02', 'bb'), ('03', 'bb'), ('12', 'ba'), ('13', 'ab')], [('02', 'bb'), ('03', 'bb'), ('12', 'ba'), ('13', 'ba')], [('02', 'bb'), ('03', 'bb'), ('12', 'ba'), ('13', 'bb')], [('02', 'bb'), ('03', 'bb'), ('12', 'bb'), ('13', 'aa')], [('02', 'bb'), ('03', 'bb'), ('12', 'bb'), ('13', 'ab')], [('02', 'bb'), ('03', 'bb'), ('12', 'bb'), ('13', 'ba')], [('02', 'bb'), ('03', 'bb'), ('12', 'bb'), ('13', 'bb')]]
types = [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0]
priors = [-0.9178860550328204, -10.749415928290118, -10.749415928290118, -11.272664072079987, -10.749415928290118, -10.749415928290118, -16.95425710594061, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -11.272664072079987, -17.294055179550075, -17.294055179550075, -11.272664072079987, -10.749415928290118, -10.749415928290118, -16.95425710594061, -17.294055179550075, -10.749415928290118, -10.749415928290118, -16.95425710594061, -17.294055179550075, -16.95425710594061, -16.95425710594061, -16.95425710594061, -12.460704095246543, -17.294055179550075, -17.294055179550075, -20.83821243446749, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -16.95425710594061, -16.95425710594061, -16.95425710594061, -12.460704095246543, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -17.294055179550075, -20.83821243446749, -17.294055179550075, -17.294055179550075, -11.272664072079987, -17.294055179550075, -17.294055179550075, -11.272664072079987, -17.294055179550075, -17.294055179550075, -20.83821243446749, -17.294055179550075, -17.294055179550075, -20.83821243446749, -17.294055179550075, -17.294055179550075, -11.272664072079987, -17.294055179550075, -17.294055179550075, -11.272664072079987, -10.749415928290118, -10.749415928290118, -16.95425710594061, -17.294055179550075, -10.749415928290118, -10.749415928290118, -16.95425710594061, -17.294055179550075, -16.95425710594061, -16.95425710594061, -16.95425710594061, -20.83821243446749, -17.294055179550075, -17.294055179550075, -12.460704095246543, -17.294055179550075, -10.749415928290118, -10.749415928290118, -16.95425710594061, -17.294055179550075, -10.749415928290118, -2.304180416152711, -11.272664072079987, -10.749415928290118, -16.95425710594061, -11.272664072079987, -11.272664072079987, -16.95425710594061, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -16.95425710594061, -16.95425710594061, -16.95425710594061, -20.83821243446749, -16.95425710594061, -11.272664072079987, -11.272664072079987, -16.95425710594061, -16.95425710594061, -11.272664072079987, -11.272664072079987, -16.95425710594061, -20.83821243446749, -16.95425710594061, -16.95425710594061, -16.95425710594061, -17.294055179550075, -17.294055179550075, -12.460704095246543, -17.294055179550075, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -20.83821243446749, -16.95425710594061, -16.95425710594061, -16.95425710594061, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -16.95425710594061, -16.95425710594061, -16.95425710594061, -20.83821243446749, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -17.294055179550075, -12.460704095246543, -17.294055179550075, -17.294055179550075, -16.95425710594061, -16.95425710594061, -16.95425710594061, -20.83821243446749, -16.95425710594061, -11.272664072079987, -11.272664072079987, -16.95425710594061, -16.95425710594061, -11.272664072079987, -11.272664072079987, -16.95425710594061, -20.83821243446749, -16.95425710594061, -16.95425710594061, -16.95425710594061, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -16.95425710594061, -11.272664072079987, -11.272664072079987, -16.95425710594061, -10.749415928290118, -11.272664072079987, -2.304180416152711, -10.749415928290118, -17.294055179550075, -16.95425710594061, -10.749415928290118, -10.749415928290118, -17.294055179550075, -12.460704095246543, -17.294055179550075, -17.294055179550075, -20.83821243446749, -16.95425710594061, -16.95425710594061, -16.95425710594061, -17.294055179550075, -16.95425710594061, -10.749415928290118, -10.749415928290118, -17.294055179550075, -16.95425710594061, -10.749415928290118, -10.749415928290118, -11.272664072079987, -17.294055179550075, -17.294055179550075, -11.272664072079987, -17.294055179550075, -17.294055179550075, -20.83821243446749, -17.294055179550075, -17.294055179550075, -20.83821243446749, -17.294055179550075, -17.294055179550075, -11.272664072079987, -17.294055179550075, -17.294055179550075, -11.272664072079987, -17.294055179550075, -17.294055179550075, -20.83821243446749, -17.294055179550075, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -12.460704095246543, -16.95425710594061, -16.95425710594061, -16.95425710594061, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -20.83821243446749, -17.294055179550075, -17.294055179550075, -12.460704095246543, -16.95425710594061, -16.95425710594061, -16.95425710594061, -17.294055179550075, -16.95425710594061, -10.749415928290118, -10.749415928290118, -17.294055179550075, -16.95425710594061, -10.749415928290118, -10.749415928290118, -11.272664072079987, -17.294055179550075, -17.294055179550075, -11.272664072079987, -17.294055179550075, -10.749415928290118, -16.95425710594061, -10.749415928290118, -17.294055179550075, -16.95425710594061, -10.749415928290118, -10.749415928290118, -11.272664072079987, -10.749415928290118, -10.749415928290118, -0.9178860550328204]


types = np.array(types)
no_of_each_type = np.bincount(types)
print('')
print("no_of_each_type ACCORDING TO SIMLANG CODE is:")
print(no_of_each_type)

class_per_lang = classify_all_languages(all_possible_languages)
print('')
print('')
# print("class_per_lang is:")
# print(class_per_lang)
no_of_each_class = np.bincount(class_per_lang.astype(int))
print('')
print("no_of_each_class is:")
print(no_of_each_class)


no_of_each_type iIN SIM is:
[  4  16 228   8]



no_of_each_class is:
[  4  20 228   4]


Hmmm, that gives us slightly different numbers! Is that caused by a problem in our create_all_languages() function, or in our classify_lang() function? To find out, let's compare our list of all languages to that from SimLang lab 21:

In [13]:
# First, we need to change the way we represent the list of all languages to match
# that of lab 21:

def transform_all_languages_to_simlang_format(language_list):
    """
    Takes a list of languages as represented by me (with only the forms listed
    for each language, assuming the meaning for each form is specified by the
    form's index), and turning it into a list of languages as represented in
    SimLang lab 21 (which in turn is based on Kirby et al., 2015), in which a
    <meaning, form> pair forms a tuple, and four of those tuples in a list form
    a language

    :param language_list: list of all languages
    :type language_list: list
    :returns: list of the input languages in the format of SimLang lab 21
    :rtype: list
    """
    all_langs_as_in_simlang = []
    for l in range(len(all_possible_languages)):
        lang_as_in_simlang = [(meanings[x], all_possible_languages[l][x]) for x in range(len(meanings))]
        all_langs_as_in_simlang.append(lang_as_in_simlang)
    return all_langs_as_in_simlang


all_langs_as_in_simlang = transform_all_languages_to_simlang_format(all_possible_languages)
print('')
print('')
# print("all_langs_as_in_simlang is:")
# print(all_langs_as_in_simlang)
print("len(all_langs_as_in_simlang) is:")
print(len(all_langs_as_in_simlang))
print("len(all_langs_as_in_simlang[0]) is:")
print(len(all_langs_as_in_simlang[0]))
print("len(all_langs_as_in_simlang[0][0]) is:")
print(len(all_langs_as_in_simlang[0][0]))



len(all_langs_as_in_simlang) is:
256
len(all_langs_as_in_simlang[0]) is:
4
len(all_langs_as_in_simlang[0][0]) is:
2


In [14]:
def check_all_lang_lists_against_each_other(language_list_a, language_list_b):
    """
    Takes two lists of languages of the same length and format, and checks for each languages in language_list_a,
    whether it is also present in language_list_b.

    :param language_list_a: list of languages represented as in the SimLang lab 21 code, where each language is a list
    of 4 tuples, where each tuple consists of a meaning and its corresponding form.
    :param language_list_b: list of languages of same format as language_list_b
    :return: a list of binary values of the same length as language_list_a, where 1. means "is present in
    language_list_b", and 0. means "not present".
    """
    if len(language_list_a) != len(language_list_b):
        raise ValueError("The two language lists should be of the same size")
    checks_per_lang = np.zeros(len(language_list_a))
    for i in range(len(language_list_a)):
        for j in range(len(language_list_b)):
            if language_list_a[i] == language_list_b[j]:
                checks_per_lang[i] = 1.
    return checks_per_lang


checks_per_language = check_all_lang_lists_against_each_other(all_langs_as_in_simlang, languages)
print('')
print('')
print("checks_per_language is:")
print(checks_per_language)
print("np.sum(checks_per_language) is:")
print(np.sum(checks_per_language))



checks_per_language is:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
np.sum(checks_per_language) is:
256.0


Ok, this shows that for each language in the list of all_possible_languages generated by my own code, there is a corresponding languages in the code from SimLang lab 21, so instead there must be something wrong with the way I categorise the languages. Firstly, it looks like my classify_language() function overestimates the number of compositional languages. So let's first have a look at which languages it classifies as compositional:

In [9]:


compositional_langs_indices_my_code = np.where(class_per_lang==3)[0]
print('')
print('')
print("compositional_langs_indices_my_code MY CODE are:")
print(compositional_langs_indices_my_code)
print("len(compositional_langs_indices_my_code) MY CODE are:")
print(len(compositional_langs_indices_my_code))


for index in compositional_langs_indices_my_code:
    print('')
    print("index MY CODE is:")
    print(index)
    print("all_possible_languages[index] MY CODE is:")
    print(all_possible_languages[index])


# And now let's do the same for the languages from SimLang Lab 21:

compositional_langs_indices_simlang = np.where(np.array(types)==3)[0]
print('')
print('')
print("compositional_langs_indices_simlang SIMLANG CODE are:")
print(compositional_langs_indices_simlang)
print("len(compositional_langs_indices_simlang) SIMLANG CODE are:")
print(len(compositional_langs_indices_simlang))


for index in compositional_langs_indices_simlang:
    print('')
    print("index SIMLANG CODE is:")
    print(index)
    print("languages[index] SIMLANG CODE is:")
    print(languages[index])

NameError: name 'class_per_lang' is not defined

Hmm, so it looks like instead of there being a bug in my code, there's actually a bug in the SimLang lab 21 code (or rather, in the code that generated the list of types that was copied into SimLang lab 21

Now let's implement the actual production function. Here is the equation from Kirby et al. (2015) again:

$$
P(f \mid l, t) \propto \Bigg\{
\begin{array}{ll}
(\frac{1}{a})^\gamma \, (1-\epsilon) \; \; \textrm{if} \; t \; \textrm{is mapped to} \; f \; \textrm{in} \; l\\ 
\frac{\epsilon}{|F|-1} \quad \quad \quad \textrm{if} \; t \; \textrm{is not mapped to} \; f \; \textrm{in} \; l
\end{array}
$$

where $f$ stands for form (in the sense of a complete signal, such as *aa*), $l$ stands for language, $t$ stands for topic, and $\epsilon$ stands for production error.

In [15]:
# first we need to write a quick function that removes every instance of a given element from a list (to use for
# removing the 'correct' forms from a list of possible forms for a given topic:
def remove_all_instances(my_list, element_to_be_removed):
    """
    Takes a list, and removes all instances of a given element from it
    
    :param my_list: a list
    :param element_to_be_removed: the element to be removed; can be of any type
    :return: the list with all instances of the target element removed
    """
    i = 0  # loop counter
    length = len(my_list)  # list length
    while (i < len(my_list)):
        if (my_list[i] == element_to_be_removed):
            my_list.remove(my_list[i])
            # as an element is removed
            # so decrease the length by 1
            length = length - 1
            # run loop again to check element
            # at same index, when item removed
            # next item will shift to the left
            continue
        i = i + 1
    return my_list


# and now for the actual production function:
def production_kirby_et_al(language, topic):
    """
    A reproduction of the production function of Kirby et al. (2015)
    
    :param language: list of forms that has same length as list of meanings (global variable), where each form is 
    mapped to the meaning at the corresponding index
    :param topic: the index of the topic (corresponding to an index in the globally defined meaning list) that the 
    speaker intends to communicate
    :return: a form (i.e. utterance)
    """
    print('')
    print('This is the production() function at work:')
    print('')
    print("language is:")
    print(language)
    print('')
    print("meanings are:")
    print(meanings)
    for m in range(len(meanings)):
        if meanings[m] == topic:
            topic_index = m
    print('')
    print("topic_index is:")
    print(topic_index)
    correct_form = language[topic_index]
    print("correct_form is:")
    print(correct_form)
    error_forms = list(language)
    error_forms = remove_all_instances(error_forms, correct_form)
    if len(error_forms) == 0:  # if the list of error_forms is empty because the language is degenerate
        error_forms = language  # simply choose an error_form from the whole language
    print("error_forms is:")
    print(error_forms)
    print("Check that removing the correct form didn't mess up the language:")
    print("language is:")
    print(language)



print('')
print('')
print('Produce a signal using the example compositional language:')
utterance = production_kirby_et_al(compositional_lang, "02")
print(utterance)



Produce a signal using the example compositional language:

This is the production() function at work:

language is:
('aa', 'ab', 'ba', 'bb')

meanings are:
['02', '03', '12', '13']

topic_index is:
0
correct_form is:
aa
error_forms is:
['ab', 'ba', 'bb']
Check that removing the correct form didn't mess up the language:
language is:
('aa', 'ab', 'ba', 'bb')
None


## Now let's implement noisy production:

### References  
Kirby, S., Tamariz, M., Cornish, H., & Smith, K. (2015). Compression and communication in the cultural evolution of linguistic structure. Cognition, 141, 87–102. https://doi.org/10.1016/j.cognition.2015.03.016